<a href="https://colab.research.google.com/github/Guilli12pm/ML_proj/blob/master/project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning project

In [0]:
!git clone https://github.com/Guilli12pm/ML_proj

fatal: destination path 'ML_proj' already exists and is not an empty directory.


In [0]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!ls ML_proj

check_photos.py  haarcascade_eye_tree_eyeglasses.xml  next_ite
create_data.py	 haarcascade_eye.xml		      __pycache__
Database	 haarcascade_frontalface_default.xml  README.md
eyes_check.py	 haarcascade_smile.xml		      reset.py


In [0]:
print(1)